# Programação para Sensoriamento Remoto
<hr style="border:2px solid #0077b9;">

- Professores:
  - Gilberto Ribeiro de Queiroz
  - Thales Sehn Körting
  - Eugênio Almeida
- Colaboradoes:
  - Vitor Gomes

## Objetivo
<hr style="border:2px solid #0077b9;">

Criar uma grade numérica com valores acumulados do número de focos de queimadas no ano de 2016.

## Dados de Entrada
<hr style="border:2px solid #0077b9;">

<table style="font-size: 20px;">
<caption>Arquivo ESRI Shapefile contendo os focos de queimadas do ano de 2016.</caption>
<tr>
  <td>
    <img src="../img/focos-2016.png" alt="Focos Brasil no ano de 2016" style="height: 300px;" align="middle"/>
  </td>
  <td>
    <table>
      <caption>Sensores/Satélites</caption>
      <tbody>
      <tr>
        <td style="text-align: left">AQUA_M-M</td>
        <td style="text-align: left">NOAA-15</td>    
        <td style="text-align: left">NOAA-19D</td>
      </tr>
      <tr>
        <td style="text-align: left">AQUA_M-T</td>
        <td style="text-align: left">NOAA-15D</td>    
        <td style="text-align: left">NPP</td>  
      </tr>
      <tr>
        <td style="text-align: left">GOES-13`</td>
        <td style="text-align: left">NOAA-18</td>
        <td style="text-align: left">NPP_375</td> 
      </tr>
      <tr>
        <td style="text-align: left">METOP-B</td>    
        <td style="text-align: left">NOAA-18D</td>
        <td style="text-align: left">TERRA_M-M</td> 
      </tr>
      <tr>
        <td style="text-align: left">MSG-03</td>    
        <td style="text-align: left">NOAA-19</td>
        <td style="text-align: left">TERRA_M-T</td>  
      </tr>
      </tbody>
    </table>
  </td>
</tr>
</table>

## Estratégia
<hr style="border:2px solid #0077b9;">

A dimensão e a resolução espacial da grade serão dadas por:
- dimensões:
```python
{ 'cols': 1200, 'rows': 1400 }
```

- resolução espacial :
```python
{ 'xres': 0.05, 'yres': 0.05 }
```

- extensão espacial:
```python
{
  'xmin': -89.975, 'xmax': -29.975,
  'ymin': -59.975, 'ymax': 10.025
}
```

## Programa
<hr style="border:2px solid #0077b9;">

In [ ]:
# encoding: utf-8

### Carga de Bibliotecas
<hr style="border:2px solid #0077b9;">

In [ ]:
import sys


# Carrega a biblioteca OS que fornece funcionalidades
# associadas ao Sistema Operacional
import os


# Carrega a biblioteca NumPy para criação de matrizes
import numpy as np

Carrega a biblioteca GDAL com seus módulos:
- Raster
- Vetorial
- Sistema de Referência Espacial

In [ ]:
try:
    from osgeo import gdal, ogr, osr
except:
    sys.exit("Erro: a biblioteca GDAL não foi encontrada!")

Carrega as rotinas auxiliares que encontram-se definidas na mesma pasta deste programa:

In [ ]:
from utils import *

### Configurações GDAL
<hr style="border:2px solid #0077b9;">

Habilita o lançamento de exceções caso alguma rotina da GDAL/OGR encontre algum erro

In [ ]:
gdal.UseExceptions()
ogr.UseExceptions()
osr.UseExceptions()

### Definição de constantes globais do programa
<hr style="border:2px solid #0077b9;">

As associações abaixo irão facilitar modificações no programa:

In [ ]:
vector_file = "/home/gribeiro/Dados/Queimadas/focos/focos-2016.shp"

vector_file_base_name = os.path.basename(vector_file)

layer_name = os.path.splitext(vector_file_base_name)[0]

In [ ]:
spatial_extent = { 'xmin': -89.975, 'ymin': -59.975,
                   'xmax': -29.975, 'ymax': 10.025 }

spatial_resolution = { 'x': 0.05, 'y': 0.05 }

grid_dimensions = { 'cols': 1200, 'rows': 1400 }

In [ ]:
file_format = "GTiff"

output_file_name = "/home/gribeiro/Dados/Queimadas/focos/grade-2016.tiff"

### Abrindo arquivo vetorial com focos de queimada
<hr style="border:2px solid #0077b9;">


Abre o arquivo ESRI Shapefile contendo os focos de queimada do ano de 2016

In [ ]:
shp_focos = ogr.Open(vector_file)

if shp_focos is None:
    sys.exit("Erro: não foi possível abrir o arquivo '{0}'.".format(vector_file))

Recupera a camada de informação com os focos:

In [ ]:
layer_focos = shp_focos.GetLayer(layer_name)

if layer_focos is None:
    sys.exit("Erro: não foi possível acessar a camada '{0}' no arquivo {1}!".format(layer_name, vector_file))

### Criando uma matriz numérica
<hr style="border:2px solid #0077b9;">

In [ ]:
matriz = np.zeros((grid_dimensions['rows'], grid_dimensions['cols']),
                  np.int16)

### Calculando o número de focos associado a cada célula
<hr style="border:2px solid #0077b9;">

Itera por cada um dos focos calculando sua localização na grade:

In [ ]:
for foco in layer_focos:
    location = foco.GetGeometryRef()

    col, row = Geo2Grid(location, grid_dimensions,
                        spatial_resolution, spatial_extent)

    matriz[row, col] += 1

### Criando o raster de saída
<hr style="border:2px solid #0077b9;">

Cria uma fonte de dados de destino para escrever o raster em disco no formato GeoTIFF

In [ ]:
driver = gdal.GetDriverByName(file_format)

if driver is None:
    sys.exit("Erro: não foi possível identificar o driver '{0}'.".format(file_format))

raster = driver.Create(output_file_name,
                       grid_dimensions['cols'], grid_dimensions['rows'],
                       1, gdal.GDT_UInt16)

if raster is None:
    sys.exit("Erro: não foi possível criar o arquivo '{0}'.".format(output_file_name))

Define os parâmetros de transformação de coordenadas:

In [ ]:
raster.SetGeoTransform((spatial_extent['xmin'], spatial_resolution['x'], 0,
                        spatial_extent['ymax'], 0, -spatial_resolution['y']))

Usa as informações do sistema de coordenadas espacial do layer de focos na definição da grade de saída:

In [ ]:
srs_focos = layer_focos.GetSpatialRef()

raster.SetProjection(srs_focos.ExportToWkt())

Acessa o objeto associado a primeira banda do raster e escreve o array NumPy na banda da GDAL:

In [ ]:
band = raster.GetRasterBand(1)

band.WriteArray(matriz, 0, 0)

band.FlushCache()

Garante que toda a estrutura do raster da GDAL foi desalocada:

In [ ]:
raster = None
del raster, band

## Exercícios
<hr style="border:2px solid #0077b9;">

Alterar programa desse notebook python para que ele seja capaz de criar uma grade para cada um dos seguintes sensores/satélites:
```python
{ 'TERRA_M-M', 'TERRA_M-T', 'AQUA_M-T', 'AQUA_M-M' }
```

Dica: utilize os tipos **set** e **dicionários**!

```python
Xgeo = GT(0) + Xpixel*GT(1) + Ypixel*GT(2)
Ygeo = GT(3) + Xpixel*GT(4) + Ypixel*GT(5)
```